# 银行卡支付欺诈检测

在此部分，我们将查看信用卡欺诈检测数据集，并构建一个二元分类模型，该模型可以根据提供的历史数据，将交易识别为欺诈性交易或有效交易。在 [2016 年的一项研究中](https://nilsonreport.com/upload/content_promo/The_Nilson_Report_10-17-2016.pdf)，人们发现信用卡欺诈给全球带来了 200 亿美元以上的损失。准确检测欺诈行为是一个很活跃的研究领域。

<img src=notebook_ims/fraud_detection.png width=50% />

### 带标签数据

你可以从 [Kaggle](https://www.kaggle.com/mlg-ulb/creditcardfraud/data) 下载支付欺诈数据集（Dal Pozzolo 等人，2015 年）。这个数据集包含成千上万的信用卡交易特征和标签，每个交易都标记为欺诈性交易或有效交易。在此 notebook 中，我们将根据这些交易的特征训练一个模型，以便未来能够预测有风险或欺诈性交易。

### 二元分类

因为我们有真实标签可以参考，所以将采用**监督式学习**方法，并训练二元分类器将数据分成两种交易类别之一：欺诈性交易或有效交易。我们将用训练数据训练模型，并看看模型泛化到测试数据的效果。

该 notebook 将分成以下几个步骤：
* 加载和探索数据
* 将数据拆分为训练集和测试集
* 定义和训练一个二元分类器 LinearLearner
* 改进模型
* 评估和比较模型测试效果

### 改进模型

这个 notebook 侧重于改进模型，如[这篇 SageMaker 博文](https://aws.amazon.com/blogs/machine-learning/train-faster-more-flexible-models-with-amazon-sagemaker-linear-learner/)所述。我们将完成以下两项任务：

1. **优化模型的超参数**，并根据某个指标改进模型，例如提高召回率或精确率。
2. **解决类别不平衡性问题**，即一个类别的训练样本比另一个类别的样本多得多（在此示例中，有效交易样本比欺诈性交易样本多得多）。

---

首先，导入常规资源。

In [ ]:
import io
import os
import matplotlib.pyplot as plt
import numpy as np 
import pandas as pd 

import boto3
import sagemaker
from sagemaker import get_execution_role

%matplotlib inline

我将在下个单元格中存储 **SageMaker 变量**：
* sagemaker_session：我们训练模型时将使用的 SageMaker 会话。
* bucket：存储数据时使用的默认 S3 存储桶的名称。
* role：定义数据和模型权限的 IAM 角色。

In [ ]:
# sagemaker session, role
sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()

# S3 bucket name
bucket = sagemaker_session.default_bucket()


## 加载和探索数据

接下来，加载数据并解压缩 `creditcardfraud.zip` 文件中的数据。此目录下有一个 csv 文件 `creditcard.csv`，其中包含所有交易数据。

与之前的 notebook 一样，一定要看看数据的分布，这样我们才知道如何开发欺诈检测模型。我们想要知道：数据集中的数据点数量、特征数量和类型，以及每个类别（有效或欺诈性交易）的数据分布。

In [ ]:
# only have to run once
!wget https://s3.amazonaws.com/video.udacity-data.com/topher/2019/January/5c534768_creditcardfraud/creditcardfraud.zip
!unzip creditcardfraud


In [ ]:
# read in the csv file
local_data = 'creditcard.csv'

# print out some data
transaction_df = pd.read_csv(local_data)
print('Data shape (rows, cols): ', transaction_df.shape)
print()
transaction_df.head()

### 练习：计算欺诈性数据的百分比

看看每个类别（有效和欺诈性）的交易数据分布。

请完成以下函数 `fraudulent_percentage`。统计每个类别的数据点数量，并计算欺诈性数据点的百分比。

In [ ]:
# Calculate the fraction of data points that are fraudulent
def fraudulent_percentage(transaction_df):
    '''Calculate the fraction of all data points that have a 'Class' label of 1; fraudulent.
       :param transaction_df: Dataframe of all transaction data points; has a column 'Class'
       :return: A fractional percentage of fraudulent data points/all points
    '''
    
    # your code here
    
    pass


通过调用函数并输出结果，测试代码。

In [ ]:
# call the function to calculate the fraud percentage
fraud_percentage = fraudulent_percentage(transaction_df)

print('Fraudulent percentage = ', fraud_percentage)
print('Total # of fraudulent pts: ', fraud_percentage*transaction_df.shape[0])
print('Out of (total) pts: ', transaction_df.shape[0])


### 练习：将数据划分为训练集和测试集

在此示例中，我们想要评估欺诈分类器的性能；使用训练数据训练模型，然后使用在训练过程中没见过的测试数据测试模型。我们需要将数据划分为训练集和测试集。

请完成以下函数 `train_test_split`。此函数应该：
* 随机重排交易数据
* 根据参数 `train_frac` 将数据分成两部分
* 获取训练和测试特征及标签
* 返回元组：(train_features, train_labels), (test_features, test_labels)

In [ ]:
# split into train/test
def train_test_split(transaction_df, train_frac= 0.7, seed=1):
    '''Shuffle the data and randomly split into train and test sets;
       separate the class labels (the column in transaction_df) from the features.
       :param df: Dataframe of all credit card transaction data
       :param train_frac: The decimal fraction of data that should be training data
       :param seed: Random seed for shuffling and reproducibility, default = 1
       :return: Two tuples (in order): (train_features, train_labels), (test_features, test_labels)
       '''
    
    # shuffle and split the data
    train_features = None
    train_labels = None
    test_features = None
    test_labels = None
    
    return (train_features, train_labels), (test_features, test_labels)


### 测试单元格

在以下单元格中，我创建了训练数据和测试数据，并检查结果是否合理。以下测试会测试上述函数是否将数据划分成规定数量的数据点，并且标签的确是类别标签 (0, 1)。

In [ ]:
# get train/test data
(train_features, train_labels), (test_features, test_labels) = train_test_split(transaction_df, train_frac=0.7)

In [ ]:
# manual test

# for a split of 0.7:0.3 there should be ~2.33x as many training as test pts
print('Training data pts: ', len(train_features))
print('Test data pts: ', len(test_features))
print()

# take a look at first item and see that it aligns with first row of data
print('First item: \n', train_features[0])
print('Label: ', train_labels[0])
print()

# test split
assert len(train_features) > 2.333*len(test_features), \
        'Unexpected number of train/test points for a train_frac=0.7'
# test labels
assert np.all(train_labels)== 0 or np.all(train_labels)== 1, \
        'Train labels should be 0s or 1s.'
assert np.all(test_labels)== 0 or np.all(test_labels)== 1, \
        'Test labels should be 0s or 1s.'
print('Tests passed!')

---
# 建模

上传训练数据后，下面定义和训练模型。

在此 notebook 中，你将定义和训练 SageMaker 的内置算法 [LinearLearner](https://sagemaker.readthedocs.io/en/stable/linear_learner.html)。 

LinearLearner 有两个主要用途：
1. 完成回归任务，即用一条线拟合一些数据点，你希望模型能根据某些数据点生成预测输出值（例如根据平方面积预测房价）
2. 用于二元分类，有一条线能区分两类数据并有效地输出标签；数据点在这条线上方的类别标签为 1，数据点在这条线上或下面的类别标签为 0。

<img src='notebook_ims/linear_separator.png' width=50% />

在此 notebook 中，我们将用于第二种情形，并训练 LinearLearner 将数据分成两个类别：有效或欺诈性数据。

### 练习：创建 LinearLearner 评估器

你已经练习过在 SageMaker 中实例化内置模型。所有评估器都需要传入一些构造函数参数。看看你能否仅参考 [LinearLearner 文档](https://sagemaker.readthedocs.io/en/stable/linear_learner.html)实例化一个 LinearLearner 评估器。此评估器有很多参数，但是并非必须都要指定。我的建议是从简单的模型开始，并尽量利用默认值。稍后，我们将讨论一些具体的超参数及其用例。

#### 实例类型

建议使用免费套餐里提供的实例“ml.c4.xlarge”训练模型，并使用“ml.t2.medium”部署模型。

In [ ]:
# import LinearLearner
from sagemaker import LinearLearner

# instantiate LinearLearner


### 练习：将数据转换成 RecordSet 格式

接下来，将训练特征和标签转换成浮点值 numpy 数组，为内置模型准备好数据。然后使用 [record_set 函数](https://sagemaker.readthedocs.io/en/stable/linear_learner.html#sagemaker.LinearLearner.record_set)将数据变成 RecordSet，并为训练做好准备。

In [ ]:
# create RecordSet of training data
formatted_train_data = None

### 练习：训练评估器

实例化评估器后，通过调用 `.fit()` 训练该评估器，并传入特殊格式的训练数据。

In [ ]:
%%time 
# train the estimator on formatted training data


### 练习：部署训练过的模型

部署模型以创建预测器。我们将使用该预测器对测试数据进行预测并评估模型。

In [ ]:
%%time 
# deploy and create a predictor
linear_predictor = None

---
# 评估模型

部署模型后，可以评估模型在测试数据上的效果。

根据[预测器文档](https://sagemaker.readthedocs.io/en/stable/linear_learner.html#sagemaker.LinearLearnerPredictor)，此预测器要求传入 `ndarray` 格式的输入特征，并返回记录列表。
> "预测存储在 `Record.label` 字段的  "predicted_label" 键中。"

先测试下模型在一个测试数据点上的效果，看看生成的列表。

In [ ]:
# test one prediction
test_x_np = test_features.astype('float32')
result = linear_predictor.predict(test_x_np[0])

print(result)

### 评估辅助函数


以下函数的参数包括部署的预测器、测试特征和标签，并返回指标字典；计算真假正例和真假负例，以及召回率、精确率和准确率。

In [ ]:
# code to evaluate the endpoint on test data
# returns a variety of model metrics
def evaluate(predictor, test_features, test_labels, verbose=True):
    """
    Evaluate a model on a test set given the prediction endpoint.  
    Return binary classification metrics.
    :param predictor: A prediction endpoint
    :param test_features: Test features
    :param test_labels: Class labels for test data
    :param verbose: If True, prints a table of all performance metrics
    :return: A dictionary of performance metrics.
    """
    
    # We have a lot of test data, so we'll split it into batches of 100
    # split the test data set into batches and evaluate using prediction endpoint    
    prediction_batches = [predictor.predict(batch) for batch in np.array_split(test_features, 100)]
    
    # LinearLearner produces a `predicted_label` for each data point in a batch
    # get the 'predicted_label' for every point in a batch
    test_preds = np.concatenate([np.array([x.label['predicted_label'].float32_tensor.values[0] for x in batch]) 
                                 for batch in prediction_batches])
    
    # calculate true positives, false positives, true negatives, false negatives
    tp = np.logical_and(test_labels, test_preds).sum()
    fp = np.logical_and(1-test_labels, test_preds).sum()
    tn = np.logical_and(1-test_labels, 1-test_preds).sum()
    fn = np.logical_and(test_labels, 1-test_preds).sum()
    
    # calculate binary classification metrics
    recall = tp / (tp + fn)
    precision = tp / (tp + fp)
    accuracy = (tp + tn) / (tp + fp + tn + fn)
    
    # printing a table of metrics
    if verbose:
        print(pd.crosstab(test_labels, test_preds, rownames=['actual (row)'], colnames=['prediction (col)']))
        print("\n{:<11} {:.3f}".format('Recall:', recall))
        print("{:<11} {:.3f}".format('Precision:', precision))
        print("{:<11} {:.3f}".format('Accuracy:', accuracy))
        print()
        
    return {'TP': tp, 'FP': fp, 'FN': fn, 'TN': tn, 
            'Precision': precision, 'Recall': recall, 'Accuracy': accuracy}


### 测试结果

以下单元格将运行 `evaluate` 函数。

代码假设你已经在之前运行的单元格中定义了 `predictor`、`test_features` 和 `test_labels`。

In [ ]:
print('Metrics for simple, LinearLearner.\n')

# get metrics for linear predictor
metrics = evaluate(linear_predictor, 
                   test_features.astype('float32'), 
                   test_labels, 
                   verbose=True) # verbose means we'll print out the metrics


## 删除端点

我添加了一个便利函数，在使用完端点后，可以用它删除预测端点。评估完模型后，你应该删除模型端点。

In [ ]:
# Deletes a precictor.endpoint
def delete_endpoint(predictor):
        try:
            boto3.client('sagemaker').delete_endpoint(EndpointName=predictor.endpoint)
            print('Deleted {}'.format(predictor.endpoint))
        except:
            print('Already deleted: {}'.format(predictor.endpoint))

In [ ]:
# delete the predictor endpoint 
delete_endpoint(linear_predictor)

---

# 改进模型

默认 LinearLearner 的准确率很高，但是依然将某些欺诈性数据点和有效数据点分类错了。将 30 多个数据点分类成假负例（错误标记的欺诈性交易），并且将 30 来个数据点分类成假正例（错误标记的有效交易）。我们思考下在训练过程中，什么会导致这种行为，该如何改进。

**1. 模型优化**
* 假设我们要为银行系统设计应用，那么用户肯定不希望有任何有效交易被归类为欺诈性交易。我们希望尽量减少**假正例**（0 被分类为 1）。
* 另一方面，银行经理要求应用能够发现几乎所有的欺诈性交易，即使假正例的数量很高，那么我们希望尽量减少**假负例**。
* 要根据特定的产品需求和目标训练模型，我们不能只提高准确率。我们需要优化有助于降低假正例或假负例数量的指标。

<img src='notebook_ims/precision_recall.png' width=40% />
     
在此 notebook 中，我们将查看优化模型和做出优化决策的不同情形。

**2. 不平衡的训练数据**
* 在此 notebook 的开头，我们看到只有 0.17% 的训练数据被标记成欺诈性交易。所以即使模型将**所有**数据标记为有效交易，准确率依然很高。
* 这样可能会导致过拟合有效数据，其中包含一些**假负例**；即欺诈性数据 (1) 被错误地归类为有效交易 (0)。

我们按顺序解决这些问题；首先，调节模型并在训练过程中根据特定指标优化模型；其次，解决训练集中的类别不平衡性问题。


## 改进模型：模型优化

根据特定指标进行优化称为**模型优化**，SageMaker 提供了多种自动优化模型的方式。


### 创建 LinearLearner 并提高精确率

**情形：**
* 银行请你构建一个检测欺诈性交易的模型，并且要求准确率约为 85%。

所以我们需要构建一个有很多真正例并且尽量减少假负例的模型。这种模型需要很高的**召回率**：真正例/（真正例 + 假负例）。

为了提高特定的指标，我们可以使用 LinearLearner 参数 `binary_classifier_model_selection_criteria`，它是训练数据集的模型评估标准。要了解该参数，请参阅此 [LinearLearner 文档](https://sagemaker.readthedocs.io/en/stable/linear_learner.html#sagemaker.LinearLearner)。我们还需要进一步指定到底要达到什么样的值；要详细了解此参数，请点击[此处](https://docs.aws.amazon.com/sagemaker/latest/dg/ll_hyperparameters.html)。

该参数将假设模型在训练集上的性能与在测试集上的性能相差约 5%。例如，如果召回率约为 85%，我将召回率提高到 90%。

In [ ]:
# instantiate a LinearLearner
# tune the model for a higher recall
linear_recall = LinearLearner(role=role,
                              train_instance_count=1, 
                              train_instance_type='ml.c4.xlarge',
                              predictor_type='binary_classifier',
                              output_path=output_path,
                              sagemaker_session=sagemaker_session,
                              epochs=15,
                              binary_classifier_model_selection_criteria='precision_at_target_recall', # target recall
                              target_recall=0.9) # 90% recall


### 训练优化过的评估器

用新的优化评估器拟合特殊格式的训练数据。

In [ ]:
%%time 
# train the estimator on formatted training data
linear_recall.fit(formatted_train_data)

### 部署和评估优化过的评估器

部署并评估优化过的预测器。

我们假设对于经过优化并且提高了召回率的模型，假负例（欺诈性交易错误地被标记为有效交易）的数量应该更少；优化模型后，假负例的数量变少了吗？

In [ ]:
%%time 
# deploy and create a predictor
recall_predictor = linear_recall.deploy(initial_instance_count=1, instance_type='ml.t2.medium')

In [ ]:
print('Metrics for tuned (recall), LinearLearner.\n')

# get metrics for tuned predictor
metrics = evaluate(recall_predictor, 
                   test_features.astype('float32'), 
                   test_labels, 
                   verbose=True)

## 删除端点

与之前一样，评估完模型后，应该删除端点。我在下面使用了之前定义的 `delete_endpoint` 辅助函数。

In [ ]:
# delete the predictor endpoint 
delete_endpoint(recall_predictor)

---
## 改进模型：解决类别不平衡性问题

我们的模型经过优化后召回率更高，减少了**假负例**的数量。之前，我们提到类别不平衡性可能会使模型出现偏差，预测所有交易都是有效交易，导致假负例和真负例的数量更多。所以如果解决这种不平衡性，模型将得到进一步优化。

为了在训练二元分类器的过程中考虑类别不平衡性问题，我们可以使用 LinearLearner 的超参数 `positive_example_weight_mult`，它表示在训练二元分类器时分配给正样本（1，欺诈性数据）的权重。负样本（0，有效数据）的权重固定为 1。

### 练习：创建一个设定 `positive_example_weight_mult` 参数的 LinearLearner

除了提高模型的召回率（可以使用 `linear_recall` 作为起点）之外，应该添加一个考虑到类别不平衡性问题的参数。以下内容摘自超参数 `positive_example_weight_mult` 的[参考文档](https://docs.aws.amazon.com/sagemaker/latest/dg/ll_hyperparameters.html)：
> "如果你希望算法选择一个权重，使分类负样本与正样本的误差对训练损失的影响一样，请指定 `balanced`。"

还可以输入特定的浮点值，在这种情形下，正样本的权重应该比负样本的高，因为正样本更少。

In [ ]:
# instantiate a LinearLearner

# include params for tuning for higher recall
# *and* account for class imbalance in training data
linear_balanced = None


### 练习：训练平衡评估器

用新的平衡评估器拟合特殊格式的训练数据。

In [ ]:
%%time 
# train the estimator on formatted training data


### 练习：部署并评估平衡评估器

部署并评估平衡预测器。结果符合你的预期吗？

In [ ]:
%%time 
# deploy and create a predictor
balanced_predictor = None

In [ ]:
print('Metrics for balanced, LinearLearner.\n')

# get metrics for balanced predictor
metrics = evaluate(balanced_predictor, 
                   test_features.astype('float32'), 
                   test_labels, 
                   verbose=True)

## 删除端点

评估完模型后，应该删除端点。

In [ ]:
# delete the predictor endpoint 
delete_endpoint(balanced_predictor)

指标变化注意事项：

上述模型在召回率固定为约 90% 的情况下，经过优化尽量提高精确率。召回率在训练过程中固定为 90%，但是当我们将训练过的模型应用到测试数据集上时，召回率可能会变化。

---
## 模型设计
你已经知道如何优化和平衡 LinearLearner。下面请创建、训练和部署你自己的模型。这道练习属于开放式练习，使你有机会实践设计和部署模型的步骤。

### 练习：根据指定情形训练和部署一个 LinearLearner，并设定相应的超参数。

**情形：**
* 银行要求你构建一个能帮助提供良好用户体验的模型；用户应该最多只有约 15% 的有效交易被标记为欺诈性交易。

这就需要你做出设计决策：对于上述情形，在训练过程中你应该力求优化什么指标（和值）？

模型在训练集上的效果与在测试集上的效果可能有 5-10% 的偏差。例如，如果在训练集上的准确率为 80%，那么在测试集上的准确率约为 70-90%。

最终模型应该考虑类别不平衡性，并且相应地做出优化。

In [ ]:
%%time
# instantiate and train a LinearLearner

# include params for tuning for higher precision
# *and* account for class imbalance in training data


In [ ]:
%%time 
# deploy and evaluate a predictor


In [ ]:
## IMPORTANT
# delete the predictor endpoint after evaluation 


## 最终清理！

* 仔细检查是否删除了所有端点。
* 还建议直接在 AWS 控制台中手动删除 S3 存储桶、模型和端点配置。

你可以在[此文档](https://docs.aws.amazon.com/sagemaker/latest/dg/ex1-cleanup.html)中查看详细的清理说明。

---
# 总结

在此 notebook 中，你学习了如何在 SageMaker 中训练和部署 LinearLearner。此模型非常适合二元分类任务，并且需要作出设计决策和处理训练集中的类别不平衡性问题。

你根据机器学习工作流程的步骤，加载了一些信用卡交易数据，探索了该数据并为训练模型准备好数据。然后根据不同的设计考虑事项训练、部署和评估了多个模型。